In [1]:
# DAY01
nums = [int(x) for x in open('data/01.txt', 'r').readlines()]

reg = {}

TOTAL = 2020
for i, n in enumerate(nums):
    rest = TOTAL - n
    if rest in reg:
        print(f'{n} * {rest} = {n * rest}')
    reg[n] = i
    
for i, n1 in enumerate(nums):
    for j in range(i + 1, len(nums)):
        n2 = nums[j]
        rest = TOTAL - n1 - n2
        if rest in reg and reg[rest] > j:
            print(f'{n1} * {n2} * {rest} = {n1 * n2 * rest}')

1564 * 456 = 713184
764 * 857 * 399 = 261244452


In [2]:
# DAY02
def parse_line(line):
    parts = line.strip().split()
    counts = list(map(int, parts[0].split('-')))
    return (counts[0], counts[1], parts[1][0], parts[2])

def is_valid1(desc):
    cnt1, cnt2, ch, pwd = desc
    cnt = pwd.count(ch)
    return cnt1 <= cnt <= cnt2

def is_valid2(desc):
    cnt1, cnt2, ch, pwd = desc
    return (pwd[cnt1 - 1] == ch) ^ (pwd[cnt2 - 1] == ch)
    
passwords = list(map(parse_line, open('data/02.txt', 'r').readlines()))
res1 = sum([is_valid1(pwd) for pwd in passwords])
res2 = sum([is_valid2(pwd) for pwd in passwords])

print(f'Number of valid passwords: {res1}, {res2}')

Number of valid passwords: 548, 502


In [3]:
# DAY03
import math 

def count_trees(rows, dx, dy):
    h = len(rows)
    if h == 0:
        return 0
    w = len(rows[0])
    x, y = 0, 0
    res = 0
    while y < h:
        res += rows[y][x % w] == '#'
        x += dx
        y += dy
    return res

rows = [x.strip() for x in open('data/03.txt', 'r').readlines()]
res1 = count_trees(rows, 3, 1)
print(f'Num trees 1: {res1}')

SLOPES = [(1, 1), (3, 1), (5, 1), (7, 1), (1, 2)]
res2 = math.prod((count_trees(rows, x, y) for (x, y) in SLOPES))
print(f'Num trees 2: {res2}')

Num trees 1: 242
Num trees 2: 2265549792


In [5]:
# DAY04
import re

def read_doc(line):
    return dict([x.split(':') for x in line.split()])

REQUIRED_FIELDS = ['byr', 'iyr', 'eyr', 'hgt', 'hcl', 'ecl', 'pid']

def is_valid1(passport):
    return all(f in passport for f in REQUIRED_FIELDS)

def in_range(int_str, minv, maxv):
    try:
        return minv <= int(int_str) <= maxv
    except:
        return False

def is_valid_height(height_str):
    match = re.match(r"^(?P<in>\d+)in|(?P<cm>\d+)cm$", height_str)
    if match == None:
        return False
    hcm, hin = match.group('cm'), match.group('in')
    if hcm != None:
        return in_range(hcm, 150, 193)
    elif hin != None:
        return in_range(hin, 59, 76)
    else:
        return False

VALID_EYE_COLORS = ['amb', 'blu', 'brn', 'gry', 'grn', 'hzl', 'oth']

def is_valid2(passport):
    return is_valid1(passport) \
        and in_range(passport['byr'], 1920, 2002) \
        and in_range(passport['iyr'], 2010, 2020) \
        and in_range(passport['eyr'], 2020, 2030) \
        and passport['ecl'] in VALID_EYE_COLORS \
        and re.match(r"^(\d{9})$", passport['pid']) is not None \
        and re.match(r"^#[0-9a-f]{6}$", passport['hcl']) is not None \
        and is_valid_height(passport['hgt'])
    
lines = open('data/04.txt', 'r').read()
passports = [read_doc(line) for line in lines.split("\n\n")]

print(f'Valid passports 1: {sum(map(is_valid1, passports))}')
print(f'Valid passports 2: {sum(map(is_valid2, passports))}')

Valid passports 1: 264
Valid passports 2: 224


In [6]:
# DAY05
from functools import reduce

def get_id(seat):
    return reduce(lambda res, c: res * 2 + (c in 'BR'), seat, 0)

lines = [x.strip() for x in open('data/05.txt', 'r').readlines()]
ids = [get_id(line) for line in lines]

idset = set(ids)
seats = [id for id in range(1 << len(lines[0]))
    if id not in idset and id + 1 in idset and id - 1 in idset]

print(f'Max ID: {max(ids)}, Seat: {seats[0]}')

Max ID: 801, Seat: 597


In [7]:
# DAY06
groups = [g.split() for g in open('data/06.txt', 'r').read().split('\n\n')]

res1, res2 = 0, 0
for group in groups:
    counts = {}
    for answers in group:
        for ans in answers:
            if ans in counts:
                counts[ans] += 1
            else: 
                counts[ans] = 1
    res1 += len(counts)
    res2 += sum(counts[k] == len(group) for k in counts.keys())

print(f'Answer 1: {res1}, Answer 2: {res2}')

Answer 1: 6310, Answer 2: 3193


In [8]:
# DAY07
def parse_mapping(line):
    parts = line.split()
    bag_color = ' '.join(parts[0:2])
    res = {}
    for i in range(4, len(parts), 4):
        if parts[i] != 'no':
            res[' '.join(parts[i + 1:i + 3])] = int(parts[i])
    return (bag_color, res)

def build_back_mapping(mapping):
    res = {}
    for (bag, children) in mapping.items():
        for child in children.keys():
            if child in res:
                res[child].add(bag)
            else:
                res[child] = set([bag])
    return res

def count_reachable(root, back_mapping):
    visited = set()
    def rec(node):
        if node in visited:
            return 0
        res = 1
        if node in back_mapping:
            for child in back_mapping[node]:
                res += rec(child)
        visited.add(node)
        return res
    return rec(root)
    
def count_contains(root, mapping):
    res = 1
    for (child, count) in mapping[root].items():
        res += count * count_contains(child, mapping)
    return res
    
ROOT = "shiny gold"

lines = [g.strip() for g in open('data/07.txt', 'r').readlines()]
mapping = dict(parse_mapping(line) for line in lines)
back_mapping = build_back_mapping(mapping)

print(f'Reachable: {count_reachable(ROOT, back_mapping) - 1}')
print(f'Contains: {count_contains(ROOT, mapping) - 1}')

Reachable: 370
Contains: 29547


In [9]:
# DAY08
import enum

class Op(enum.Enum): 
    acc = 1
    nop = 2
    jmp = 3
    
def parse_op(line):
    parts = line.split()
    return (Op[parts[0]], int(parts[1]))

def eval_ops(ops, acc=0):
    n = len(ops)
    visited = [False]*n
    ip = 0
    while ip >= 0 and ip < n:
        if visited[ip]:
            return (False, acc)
        visited[ip] = True

        (op, val) = ops[ip]
        if op == Op.jmp:
            ip += val - 1
        elif op == Op.acc:
            acc += val
        ip += 1
    return (True, acc)

def try_mutate_program(ops):
    for i, (op, val) in enumerate(ops):
        prevOp = op
        if op == Op.jmp:
            op = Op.nop
        elif op == Op.nop:
            op = Op.jmp
        else:
            continue
            
        ops[i] = (op, val)
        (terminated, acc) = eval_ops(ops)
        if terminated:
            return acc
        ops[i] = (prevOp, val)
    
ops = [parse_op(g.strip()) for g in open('data/08.txt', 'r').readlines()]

print(f'Answer 1: {eval_ops(ops)[1]}, Answer 2: {try_mutate_program(ops)}')

Answer 1: 1782, Answer 2: 797


In [10]:
# DAY09
import bisect

def get_first_non_sum(nums, k):
    window = sorted(nums[:k])
    for i in range(k, len(nums)):
        l, r = 0, k - 1
        s = nums[i]
        while l < r:
            cur_sum = window[l] + window[r]
            if cur_sum < s:
                l += 1
            elif cur_sum > s:
                r -= 1
            else: 
                break
        if l == r:
            return s
        bisect.insort(window, s)
        del window[bisect.bisect_left(window, nums[i - k])]
        
def find_sum_range(nums, s):
    l, r = 0, 0
    cur_sum = nums[0]
    n = len(nums)
    while r < n - 1 and l < n - 1:
        if cur_sum < s:
            r += 1
            cur_sum += nums[r]
        elif cur_sum > s:
            cur_sum -= nums[l]
            l += 1
        else:
            break
    if l != r:
        return (l, r)
        
nums = [int(g) for g in open('data/09.txt', 'r').readlines()]

WINDOW_SIZE = 25

non_sum = get_first_non_sum(nums, WINDOW_SIZE)
print(f'First non-sum: {non_sum}')

(b, e) = find_sum_range(nums, non_sum)
print(f'Sum range: {min(nums[b:e + 1]) + max(nums[b:e + 1])}')

First non-sum: 85848519
Sum range: 13414198


In [11]:
# DAY10
nums = sorted(int(g) for g in open('data/10.txt', 'r').readlines())
nums.append(nums[-1] + 3)
nums.insert(0, 0)

diffs = []
for i, n in enumerate(nums[1:]):
    diffs.append(n - nums[i])
    
res1 = sum(map(lambda x: x == 1, diffs)) * sum(map(lambda x: x == 3, diffs))
print(f'Sum 1/3 diffs: {res1}')

n = len(nums)
counts = [-1]*n

def count_arrangements(idx):
    if idx == n - 1:
        return 1
    if counts[idx] >= 0:
        return counts[idx]
    res = 0
    i = idx
    while i < n - 1:
        i += 1
        diff = nums[i] - nums[idx]
        if diff <= 3:
            res += count_arrangements(i)
        else:
            break        
    counts[idx] = res
    return res

res2 = count_arrangements(0)
print(f'Number of arrangements: {res2}')

Sum 1/3 diffs: 2414
Number of arrangements: 21156911906816


In [12]:
# DAY11

OFFS = [[-1, -1], [0, -1], [1, -1], [1, 0], [1, 1], [0, 1], [-1, 1], [-1, 0]]

def get_neighbor_counts(seats):
    h, w = len(seats), len(seats[0])
    res = [[0]*w for i in range(h)]
    for y in range(h):
        for x in range(w):
            n = 0
            for dx, dy in OFFS:
                x1, y1 = x + dx, y + dy
                n += x1 >= 0 and x1 < w and y1 >= 0 and y1 < h \
                    and seats[y1][x1] == '#'
            res[y][x] = n
    return res

def get_visible_neighbor_counts(seats):
    h, w = len(seats), len(seats[0])
    cansee = [[0]*w for i in range(h)]
    res = [[0]*w for i in range(h)]
    
    for dx, dy in OFFS:
        for j in range(h):
            for i in range(w):
                x = i if dx <= 0 else w - i - 1
                y = j if dy <= 0 else h - j - 1
                x1, y1 = x + dx, y + dy
                if x1 >= 0 and x1 < w and y1 >= 0 and y1 < h:
                    s = seats[y1][x1]
                    if s == '.':
                        cansee[y][x] = cansee[y1][x1]
                    elif s == '#':
                        cansee[y][x] = True
                    elif s == 'L':
                        cansee[y][x] = False
                else:
                    cansee[y][x] = False
                res[y][x] += cansee[y][x]
    return res
                
def step(seats, new_seats, neighbors_fn, max_occupied):
    neighbor_counts = neighbors_fn(seats)
    
    num_changed = 0
    total_occupied = 0
    for y in range(len(seats)):
        for x in range(len(seats[y])):
            status = seats[y][x]
            n = neighbor_counts[y][x]
            if status == 'L' and n == 0:
                status = '#'
            elif status == '#' and n >= max_occupied:
                status = 'L'
            
            new_seats[y][x] = status
            num_changed += status != seats[y][x]
            total_occupied += status == '#'
    return (num_changed, total_occupied)
                
def iter_seats(seats, neighbors_fn, max_occupied):
    s1 = [line.copy() for line in seats]
    s2 = [line.copy() for line in seats]
    s = [s1, s2]

    idx = 0
    while True:
        (changed, occ) = step(s[idx], s[1 - idx], neighbors_fn, max_occupied)
        if changed == 0:
            return occ
        idx = 1 - idx
    
seats = [list(line.strip()) for line in open('data/11.txt', 'r').readlines()]
print(f'Answer 1: {iter_seats(seats, get_neighbor_counts, 4)}')
print(f'Answer 2: {iter_seats(seats, get_visible_neighbor_counts, 5)}')

Answer 1: 2211
Answer 2: 1995


In [13]:
# DAY12
import math
from functools import reduce

OFFS = {
    'N': (0, -1),
    'E': (1, 0),
    'S': (0, 1),
    'W': (-1, 0),
}

def step1(pos, command):
    x, y, angle = pos
    c, val = command
    if c == 'F':
        c = {270: 'N', 0: 'E', 90: 'S', 180: 'W'}[angle]
    if c in OFFS:
        offs = OFFS[c]
        x += offs[0] * val
        y += offs[1] * val
    else:
        if c == 'R':
            angle += val
        elif c == 'L':
            angle -= val
        angle = (angle + 360) % 360
    return (x, y, angle)

def step2(pos, command):
    x, y, wx, wy = pos
    c, val = command
    if c in OFFS:
        offs = OFFS[c]
        wx += offs[0] * val
        wy += offs[1] * val
    elif c == 'F':
        x += wx * val
        y += wy * val
    elif c in ['R', 'L']:
        angle = val
        if c == 'L':
            angle = -angle
        ar = math.radians(angle)
        ca, sa = math.cos(ar), math.sin(ar)
        wx1 = ca * wx - sa * wy
        wy1 = sa * wx + ca * wy
        wx, wy = round(wx1), round(wy1)
    return (x, y, wx, wy)
  
commands = [(line[0], int(line[1:])) for line in open('data/12.txt', 'r').readlines()]

x1, y1, angle1 = reduce(step1, commands, (0, 0, 0))
print(f'Answer 1: {abs(x1) + abs(y1)}')

x2, y2, wx2, wy2 = reduce(step2, commands, (0, 0, 10, -1))
print(f'Answer 2: {abs(x2) + abs(y2)}')

Answer 1: 508
Answer 2: 30761


In [14]:
# DAY13
def part1(periods, ts):
    min_diff = None
    min_p = 0
    res = 0
    for p in periods:
        if p is None:
            continue
        d = p - ts % p
        if min_diff == None or d < min_diff:
            min_diff = d
            res = d * p
            min_p = p
    return res

def part2(periods):
    stops = sorted(((p, i) for (i, p) in enumerate(periods) if p is not None))
    p0, i0 = stops[0]
    step = p0
    k = 0
    ts = i0
    while True:
        p1, i1 = stops[k + 1]
        if (ts + i1) % p1 == 0:
            step *= p1
            k += 1
            if k == len(stops) - 1:
                return ts
        ts += step

lines = open('data/13.txt', 'r').readlines()
ts = int(lines[0])
periods = [int(x) if x.isdigit() else None for x in lines[1].strip().split(',')]

print(f'Answer 1: {part1(periods, ts)}')
print(f'Answer 2: {part2(periods)}')

Answer 1: 153
Answer 2: 471793476184394


In [15]:
# DAY14
import re
from collections import namedtuple

MemOp = namedtuple('MemOp', 'mask ops')

def parse_ops(lines):
    cur_op = None
    for line in lines:
        if line.startswith('mask'):
            if cur_op != None:
                yield cur_op
            cur_op = MemOp(re.search('mask = (.*)$', line).group(1), [])
        elif line.startswith('mem'):
            match = re.search('mem\[(\d+)\] = (\d+)', line)
            cur_op.ops.append((int(match.group(1)), int(match.group(2)))) 
    yield cur_op
            
def apply_mask1(val, mask):
    bit = 1
    for i in range(len(mask) - 1, -1, -1):
        c = mask[i]
        if c == '0':
            val = val & ~bit
        elif c == '1':
            val = val | bit
        bit = bit << 1
    return val

def eval1(ops):
    mem = {}
    for mask, ops in mem_ops:
        for (addr, val) in ops:
            mem[addr] = apply_mask1(val, mask)
    return mem

def apply_mask2(val, mask, mask_pos=0):
    if mask_pos == len(mask):
        yield val
        return
    bit = 1 << (len(mask) - mask_pos - 1)
    c = mask[mask_pos]

    if c == '0':
        yield from apply_mask2(val, mask, mask_pos + 1)
    if c == 'X':
        val = val & ~bit
        yield from apply_mask2(val, mask, mask_pos + 1)
    if c == '1' or c == 'X':
        val = val | bit
        yield from apply_mask2(val, mask, mask_pos + 1)        
    
def eval2(ops):
    mem = {}
    for mask, ops in mem_ops:
        for (addr, val) in ops:
            for masked_addr in apply_mask2(addr, mask):
                mem[masked_addr] = val
    return mem

lines = [line.strip() for line in open('data/14.txt', 'r').readlines()]
mem_ops = list(parse_ops(lines))

print(f'Answer 1: {sum(eval1(mem_ops).values())}')
print(f'Answer 2: {sum(eval2(mem_ops).values())}')

Answer 1: 8471403462063
Answer 2: 2667858637669


In [2]:
# DAY15
def eval(seed, stop_k):
    positions = {}
    next_num = 0
    k = 1
    while True:
        num = seed[k - 1] if k <= len(seed) else next_num
        if k == stop_k:
            return num

        if num in positions:
            next_num = k - positions[num]
        else: 
            next_num = 0
            
        positions[num] = k
        k += 1

seed = [int(s) for s in open('data/15.txt', 'r').read().split(',')]

print(f'Answer 1: {eval(seed, 2020)}')
print(f'Answer 2: {eval(seed, 30000000)}')

Answer 1: 412
Answer 2: 243


In [8]:
# DAY16
from collections import namedtuple

Notes = namedtuple('Notes', 'categories own_ticket tickets')

def parse_notes(data):
    def parse_category(line):
        parts = line.split(': ')
        ranges = [tuple(map(int, r.split('-'))) for r in parts[1].split(' or ')]
        return (parts[0], ranges)

    def parse_ticket(line):
        return [int (x) for x in line.split(',')]
    
    parts = [part.strip().split('\n') for part in data.split('\n\n')]
    categories = dict(parse_category(line) for line in parts[0])
    own_ticket = parse_ticket(parts[1][1])
    tickets = [parse_ticket(t) for t in parts[2][1:]]
    return Notes(categories, own_ticket, tickets)

def is_valid_category(cat_ranges, val):
    for r in cat_ranges:
        if r[0] <= val <= r[1]:
            return True
    return False

def get_invalid_entries(notes):
    for (i, t) in enumerate(notes.tickets):
        for val in t:
            valid = False
            for cat_ranges in notes.categories.values():
                if is_valid_category(cat_ranges, val):
                    valid = True
                    break
            if not valid:
                yield (i, val)
                
def get_valid_tickets(notes, invalid_entries):
    res = []
    invalid_tickets = set(x[0] for x in invalid_entries)
    for (i, t) in enumerate(notes.tickets):
        if i not in invalid_tickets:
            res.append(t)
    return res

def get_category_ordering(notes):
    cat_ranges = list(notes.categories.values())
    cat_names = list(notes.categories.keys())
            
    n = len(cat_names) 
    cands_left = n * n
    candidates = [set(range(n)) for i in range(n)]
    
    # sift out invalid candidates
    for t in notes.tickets:
        for (i, x) in enumerate(t):
            for cand in candidates[i].copy():
                if not is_valid_category(cat_ranges[cand], x):
                    candidates[i].remove(cand)
                    cands_left -= 1
            if cands_left == n:
                break
    
    # iteratively subtract single-element candidate sets from others
    while cands_left > n:
        for c in candidates:
            if len(c) == 1:
                val = next(iter(c))
                for (i, c1) in enumerate(candidates):
                    if len(c1) > 1 and val in c1:
                        candidates[i].remove(val)
                        cands_left -= 1

    return [cat_names[next(iter(x))] for x in candidates]

def part2(notes):
    ordering = get_category_ordering(valid_notes)

    res = 1
    for (i, cat_name) in enumerate(ordering):
        if cat_name.startswith('departure'):
            res *= data[1][i]
    return res

notes = parse_notes(open('data/16.txt', 'r').read())

invalid_entries = list(get_invalid_entries(data))
print(f'Answer 1: {sum(x[1] for x in invalid_entries)}')

valid_notes = Notes(notes.categories, notes.own_ticket, get_valid_tickets(notes, invalid_entries))
print(f'Answer 2: {part2(valid_notes)}')

Answer 1: 25916
Answer 2: 2564529489989


In [6]:
# DAY17
import itertools

def count_neighbors(field, pos):
    if len(field) == 0:
        return 0
    res = 0
    for p in itertools.product(*[range(x - 1, x + 2) for x in pos]):
        if p != pos and p in field:
            res += 1
    return res

def get_bounds(field):
    if len(field) == 0:
        return None
    val = next(iter(field))
    res = [[x, x] for x in val]
    for pos in field:
        for i, x in enumerate(pos):
            res[i][0] = min(res[i][0], x)
            res[i][1] = max(res[i][1], x)
    return res

def step(field):
    bounds = get_bounds(field)
    res = set()
    for pos in itertools.product(*[range(b[0] - 1, b[1] + 2) for b in bounds]):
        n = count_neighbors(field, pos)
        if pos in field:
            if n == 2 or n == 3:
                res.add(pos)
        else:
            if n == 3:
                res.add(pos)
    return res

def init_field(seed, ndim=3):
    field = set()
    for y in range(len(seed)):
        for x in range(len(seed[y])):
            if seed[y][x] == '#':
                field.add(tuple([x, y] + [0] * (ndim - 2)))
    return field

NUM_STEPS = 6

def run(seed, ndim, num_steps=NUM_STEPS):
    field = init_field(seed, ndim)
    for i in range(num_steps):
        field = step(field)
    return len(field)

seed = [line.strip() for line in open('data/17.txt', 'r').readlines()]
print(f'Answer 1: {run(seed, 3)}')
print(f'Answer 2: {run(seed, 4)}')

Answer 1: 223
Answer 2: 1884
